In [1]:
import pandas as pd
import random
import math
import time
from concurrent.futures import ThreadPoolExecutor

def parse_list(string):
    # Use split and map efficiently
    return list(map(int, string.strip("[]").split()))

def calculate_total_value(weights, values, state, capacity):
    # Use list comprehension for weight and value summation
    total_weight = sum(weights[i] for i in range(len(state)) if state[i] == 1)
    total_value = sum(values[i] for i in range(len(state)) if state[i] == 1)
    return total_value if total_weight <= capacity else 0

class SimulatedAnnealing:
    def __init__(self, initial_temperature, cooling_rate, stopping_temperature, epochs):
        self.initial_temperature = initial_temperature
        self.cooling_rate = cooling_rate
        self.stopping_temperature = stopping_temperature
        self.epochs = epochs

    def _anneal(self, weights, values, capacity, initial_state):
        current_state = initial_state[:]
        current_value = calculate_total_value(weights, values, current_state, capacity)
        best_state = current_state[:]
        best_value = current_value
        iterations = 0
        
        temp = self.initial_temperature
        while temp > self.stopping_temperature:
            iterations += 1
            bit = random.randint(0, len(current_state) - 1)
            neighbor = current_state[:]
            neighbor[bit] = 1 - neighbor[bit]
            
            neighbor_value = calculate_total_value(weights, values, neighbor, capacity)
            delta = neighbor_value - current_value
            
            if delta > 0 or random.random() < math.exp(delta / temp):
                current_state = neighbor
                current_value = neighbor_value
                
                if neighbor_value > best_value:
                    best_state = neighbor
                    best_value = neighbor_value
            
            temp *= (1 - self.cooling_rate)
        
        return best_state, best_value, iterations

    def fit(self, weights, values, capacity):
        overall_best_state = []
        overall_best_value = 0
        total_iterations = 0
        total_runtime = 0

        initial_state = [0] * len(weights)

        for _ in range(self.epochs):
            start_time = time.time()
            best_state, best_value, iterations = self._anneal(weights, values, capacity, initial_state)
            end_time = time.time()
            runtime = end_time - start_time
            
            total_runtime += runtime
            total_iterations += iterations
            
            if best_value > overall_best_value:
                overall_best_value = best_value
                overall_best_state = best_state
        
        average_runtime = total_runtime / self.epochs
        average_iterations = total_iterations / self.epochs
        
        return overall_best_state, overall_best_value, average_iterations, average_runtime

def process_row(row, model):
    weights = parse_list(row['Weights'])
    values = parse_list(row['Prices'])
    capacity = int(row['Capacity'])
    
    return model.fit(weights, values, capacity)

def optimize_knapsack(df, model):
    overall_best_value = 0
    overall_best_state = []
    overall_best_row_index = -1

    # Initialize lists to store metrics
    best_values = []
    iterations_list = []
    runtimes = []

    with ThreadPoolExecutor() as executor:
        futures = [executor.submit(process_row, row, model) for _, row in df.iterrows()]
        for index, future in enumerate(futures):
            best_state, best_value, iterations, runtime = future.result()
            best_values.append(best_value)
            iterations_list.append(iterations)
            runtimes.append(runtime)
            if best_value > overall_best_value:
                overall_best_value = best_value
                overall_best_state = best_state
                overall_best_row_index = index

    # Calculate average values for metrics
    average_best_value = sum(best_values) / len(best_values)
    average_iterations = sum(iterations_list) / len(iterations_list)
    average_runtime = sum(runtimes) / len(runtimes)

    return overall_best_row_index, overall_best_state, overall_best_value, average_best_value, average_iterations, average_runtime

# Read data
df = pd.read_csv('/mnt/d/Algo code/knapsack_5_items.csv')

# Define the model and parameters
epochs = 5
model = SimulatedAnnealing(initial_temperature=10000, cooling_rate=0.003, stopping_temperature=1, epochs=epochs)

# Optimize knapsack problem
best_row_index, best_state, best_value, avg_best_value, avg_iterations, avg_runtime = optimize_knapsack(df, model)

# Print results
print(f"Row: {best_row_index}, Best state: {best_state}, Best value: {best_value}")
print(f"Average Best Value: {avg_best_value}")
print(f"Average Iterations: {avg_iterations}")
print(f"Average Runtime per instance (seconds): {avg_runtime}")


Row: 7229, Best state: [1, 1, 1, 1, 1], Best value: 82
Average Best Value: 30.9308
Average Iterations: 3066.0
Average Runtime per instance (seconds): 0.08463503367900883
